# Tutorial 16

## Constructive Solid Geometry, OpenCASCADE geometry kernel

Instead of constructing a model in a bottom-up fashion with Gmsh's built-in
geometry kernel, starting with version 3 Gmsh allows you to directly use
alternative geometry kernels. Here we will use the OpenCASCADE kernel.

In [1]:
import gmsh

In [2]:
gmsh.initialize()

In [3]:
gmsh.model.add("t16")

Let's build the same model as in `t5.py', but using constructive solid
geometry.
We first create two cubes:

In [4]:
gmsh.model.occ.addBox(0,0,0, 1,1,1, 1)
gmsh.model.occ.addBox(0,0,0, 0.5,0.5,0.5, 2)

2

We apply a boolean difference to create the "cube minus one eigth" shape:

In [5]:
gmsh.model.occ.cut([(3,1)], [(3,2)], 3)

(Tuple{Int32,Int32}[(3, 3)], Array{Tuple{Int32,Int32},1}[[(3, 3)], []])

Boolean operations with OpenCASCADE always create new entities. By default the
extra arguments `removeObject' and `removeTool' in `cut()' are set to `True',
which will delete the original entities.

We then create the five spheres:

In [6]:
x = 0; y = 0.75; z = 0; r = 0.09

0.09

In [7]:
holes = []
for t in 1:5
    global x, z
    x += 0.166
    z += 0.166
    gmsh.model.occ.addSphere(x,y,z,r, 3 + t)
    t = (3, 3 + t)
    push!(holes, t)
end

If we had wanted five empty holes we would have used `cut()' again. Here we
want five spherical inclusions, whose mesh should be conformal with the mesh
of the cube: we thus use `fragment()', which intersects all volumes in a
conformal manner (without creating duplicate interfaces):

In [8]:
ov = gmsh.model.occ.fragment([(3,3)], holes)
gmsh.model.occ.synchronize()

Creating entities using constructive solid geometry is very powerful, but can
lead to practical issues for e.g. setting mesh sizes at points, or identifying
boundaries.

To identify points or other bounding entities you can take advantage of the
`getEntities()', `getBoundary()' and `getEntitiesInBoundingBox()' functions:

In [9]:
lcar1 = .1
lcar2 = .0005
lcar3 = .055

0.055

Assign a mesh size to all the points:

In [10]:
ov = gmsh.model.getEntities(0);
gmsh.model.mesh.setSize(ov, lcar1);

Override this constraint on the points of the five spheres:

In [11]:
ov = gmsh.model.getBoundary(holes, false, false, true);
gmsh.model.mesh.setSize(ov, lcar3);

Select the corner point by searching for it geometrically:

In [12]:
eps = 1e-3
ov = gmsh.model.getEntitiesInBoundingBox(0.5-eps, 0.5-eps, 0.5-eps,
                                         0.5+eps, 0.5+eps, 0.5+eps, 0)
gmsh.model.mesh.setSize(ov, lcar2)

In [13]:
gmsh.model.mesh.generate(3)

Info    : Meshing 1D...
Info    : [ 10%] Meshing curve 23 (Circle)
Info    : [ 20%] Meshing curve 26 (Circle)
Info    : [ 20%] Meshing curve 29 (Circle)
Info    : [ 30%] Meshing curve 32 (Circle)
Info    : [ 40%] Meshing curve 35 (Circle)
Info    : [ 50%] Meshing curve 37 (Line)
Info    : [ 50%] Meshing curve 38 (Line)
Info    : [ 50%] Meshing curve 39 (Line)
Info    : [ 50%] Meshing curve 40 (Line)
Info    : [ 60%] Meshing curve 41 (Line)
Info    : [ 60%] Meshing curve 42 (Line)
Info    : [ 60%] Meshing curve 43 (Line)
Info    : [ 70%] Meshing curve 44 (Line)
Info    : [ 70%] Meshing curve 45 (Line)
Info    : [ 70%] Meshing curve 46 (Line)
Info    : [ 70%] Meshing curve 47 (Line)
Info    : [ 80%] Meshing curve 48 (Line)
Info    : [ 80%] Meshing curve 49 (Line)
Info    : [ 80%] Meshing curve 50 (Line)
Info    : [ 90%] Meshing curve 51 (Line)
Info    : [ 90%] Meshing curve 52 (Line)
Info    : [ 90%] Meshing curve 53 (Line)
Info    : [ 90%] Meshing curve 54 (Line)
Info    : [100%] Meshin

In [14]:
gmsh.write("t16.msh")

Info    : Writing 't16.msh'...
Info    : Done writing 't16.msh'


In [15]:
gmsh.finalize()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*